In [1]:
%matplotlib widget

from pathlib import Path
import numpy as np
import flammkuchen as fl
import pandas as pd

from fimpylab import LightsheetExperiment

from matplotlib import  pyplot as plt
import seaborn as sns
from tqdm import tqdm
sns.set(style="ticks", palette="deep")
cols = sns.color_palette()
import ipywidgets as widgets

from lotr.utils import zscore
from lotr.pca import pca_and_phase, get_fictive_heading, fictive_heading_and_fit, \
        fit_phase_neurons,qap_sorting_and_phase
from circle_fit import hyper_fit
from lotr.experiment_class import LotrExperiment

import lotr.plotting as pltltr
COLS = pltltr.COLS
from lotr.utils import interpolate, roll_columns_jit, zscore


In [2]:
def nan_phase_jumps(phase_array):
    out_array = phase_array.copy()
    out_array[1:][np.abs(np.diff(out_array)) > np.pi] = np.nan
    return out_array

In [3]:
plt.close("all")
path = Path(r"\\funes\Shared\experiments\E0071_lotr\full_ring\210314_f1\210314_f1_natmov")

traces = fl.load(path / "filtered_traces.h5", "/detr")

reg_df = fl.load(path / "motor_regressors.h5")
cc_motor = reg_df["all_bias_abs"].values
cc_motor_integr = reg_df["all_bias_abs_dfdt"].values
coords = fl.load(path / "data_from_suite2p_unfiltered.h5", "/coords")
anat = fl.load(path / "data_from_suite2p_unfiltered.h5", "/anatomy_stack")
traces[np.isnan(traces)] = 0

df = fl.load(path / "bouts_df.h5")# exp.get_bout_properties()
exp = LotrExperiment(path)
fn = int(exp.fn)
beh_df = exp.behavior_log

t_start_s = 50

t_lims = (t_start_s*exp.fn, exp.n_pts - t_start_s*exp.fn)
t_slice = slice(*t_lims)

In [15]:
fn

5

In [5]:
selected = fl.load(path / "selected.h5")
pcaed_t, phase_t, _, _ = pca_and_phase(traces[t_slice, selected].T, traces[t_slice, selected].T)
hf_c = hyper_fit(pcaed_t)
pcaed_t_all, _, _, _ = pca_and_phase(traces[t_slice, selected].T, traces[t_slice, :].T)

pcaed, phase, _, _ = pca_and_phase(traces[t_slice, selected], traces[:, selected])
mot_t_slice = slice(traces.shape[0] // 2, traces.shape[0])

In [6]:
exp = LotrExperiment(path)

In [7]:
%%time
import os
perm, com_phase = qap_sorting_and_phase(traces[:, selected], t_lims=t_lims)

phases_neuron, _ = fit_phase_neurons(traces[t_slice, selected], phase[t_slice])
perm_pca = np.argsort(phases_neuron)
os.system('say "Fit completed"')

100%|███████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 204.50it/s]

Wall time: 19.6 s


1

In [8]:
unwrapped_phase = np.unwrap(phase)
unwrapped_com_phase = np.unwrap(com_phase)
traj, params = fictive_heading_and_fit(unwrapped_phase, df, min_bias=0.1)


angles = phase_t
pca_scores = pcaed_t


sort_idxs = np.argsort(exp.rpc_angles)
sorted_traces = exp.traces[:, exp.hdn_indexes[sort_idxs]]
phase_shifts = (exp.network_phase / (2 * np.pi)) * (exp.n_hdns - 1)
shifted_traces = roll_columns_jit(sorted_traces, -np.round(phase_shifts))

In [97]:
# generating a summary figure 
fig_sum, ax_sum = plt.subplots(2, 1, figsize=(6,4.5), gridspec_kw={'height_ratios': [1,3]})

t_beh = np.asarray(beh_df["t"]*fn)
t_beh[-1]
ax_sum[0].plot(beh_df["t"]*fn, beh_df["tail_sum"], color=cols[7], label='Tail', rasterized=True)
ax_sum[0].legend(loc=2, bbox_to_anchor=(0.8, 2), fontsize=7)
ax_sum[0].set_aspect('auto')
ax_sum[0].set_xlim(0, 5000)
ax_sum[0].axis('off')

#ax_sum[1,1].axis("equal")

N_BINS_RESAMPLED = 100
resampling_base = np.linspace(-np.pi, np.pi, N_BINS_RESAMPLED)
angle_resampled_traces = np.zeros((exp.n_pts, N_BINS_RESAMPLED))
for i in range(exp.n_pts):
    angle_resampled_traces[i, :] = np.interp(
        resampling_base,
        exp.rpc_angles[sort_idxs],
        exp.traces[i, exp.hdn_indexes[sort_idxs]],
    )

phase_shifts_resamp = (exp.network_phase / (2 * np.pi)) * (N_BINS_RESAMPLED - 1)
shifted_traces_resamp = roll_columns_jit(
    angle_resampled_traces, -np.round(phase_shifts_resamp)
)

line = nan_phase_jumps(phase_shifts) + exp.n_hdns / 2
line = line - np.nanmin(line)
line /=(np.nanmax(line))
line *= np.pi * 2
line -= np.pi
t_line = np.arange(0, np.shape(line)[0]) / fn

ax_head = ax_sum[1]
ax_phase = ax_head.twinx()


#unwrapped_com_phase = line
ax_head.plot(np.arange(len(traj[:])) / fn, -traj, c=cols[8])
ax_head.set(ylabel="Heading (rad)", **pltltr.get_pi_labels(0.5, ax="y", coefs=(0, 5, 10, 15)))
ax_head.set_xlim(0, 1000)
pltltr.despine(ax_head, ["top"])
pltltr.despine(ax_phase, ["top"])
ax_head.set_ylim(-10, 50)
ax_head.yaxis.label.set_color(cols[8])
ax_head.set_xlabel('Time (s)')
ax_phase.set(ylabel="Unwrapped phase (rad)", **pltltr.get_pi_labels(0.5, ax="y", coefs=(-4, -2, 0)))
ax_phase.set_xlim(0, 1000)
ax_phase.invert_yaxis()

ax_phase.yaxis.label.set_color(cols[2])
ax_phase.set_ylim(2, -15)

fig_sum.savefig(path / 'heading_alone_220511.jpg')

ax_phase.plot(np.arange(len(unwrapped_com_phase)) / fn, -unwrapped_com_phase, c=cols[2])





fig_sum.savefig(path / 'heading_220511.jpg')
#pltltr.savefig(path / "fig4_det2.pdf")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [59]:
np.min(unwrapped_com_phase)

-0.06963709685634187

In [4]:
from scipy import signal

In [12]:
######### Fake HD cells for introduction:
fig, ax = plt.subplots(1,1, figsize=(2.5,2.5))
ax.set_ylabel('Firing rate (Hz)')
ax.set_xlabel('Heading direction (degrees)')
ax.spines['top'].set_visible(False)
ax.set_yticks([])
ax.spines['right'].set_visible(False)

c_list = ['Dodgerblue', 'blueviolet', 'orange', 'lightseagreen']
t = np.arange(0, 360)
for i in range(4):
    window = signal.gaussian(360, std=15) * 30
    window = np.roll(window, (90 * i) + 140)
    ax.plot(t, window, c=c_list[i])
    ax.set_yticks([])
    
    file_name = 'HDN_gaus_' + str(i) + '_2.jpg'
    fig.savefig(Path(r"\\funes\Shared\Hagar") / file_name, dpi=300)
fig.subplots_adjust(bottom=0.25)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …